# **TCDE**
**Mohammad Sarhangzadeh**<br>
*m.sarhangnote@gmail.com*

In [ ]:
def sphere_function(x):
    return np.sum(x**2)

def rastrigin_function(x):
    return 10 * len(x) + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))

def ackley_function(x):
    n = len(x)
    sum_sq = np.sum(x**2)
    sum_cos = np.sum(np.cos(2 * np.pi * x))
    return -20 * np.exp(-0.2 * np.sqrt(sum_sq / n)) - np.exp(sum_cos / n) + 20 + np.e

def rosenbrock_function(x):
    return np.sum(100 * (x[1:] - x[:-1]**2)**2 + (x[:-1] - 1)**2)

def griewank_function(x):
    sum_sq = np.sum(x**2) / 4000
    prod_cos = np.prod(np.cos(x / np.sqrt(np.arange(1, len(x) + 1))))
    return sum_sq - prod_cos + 1

def schwefel_function(x):
    return 418.9829 * len(x) - np.sum(x * np.sin(np.sqrt(np.abs(x))))

def levy_function(x):
    w = 1 + (x - 1) / 4
    term1 = np.sin(np.pi * w[0])**2
    term2 = np.sum((w[:-1] - 1)**2 * (1 + 10 * np.sin(np.pi * w[:-1] + 1)**2))
    term3 = (w[-1] - 1)**2 * (1 + np.sin(2 * np.pi * w[-1])**2)
    return term1 + term2 + term3

def michalewicz_function(x, m=10):
    i = np.arange(1, len(x) + 1)
    return -np.sum(np.sin(x) * (np.sin(i * x**2 / np.pi))**(2 * m))


In [ ]:
import numpy as np


def weighted_lehmer_mean(values, weights):
    num = np.sum(weights * np.square(values))
    denom = np.sum(weights * values)
    return num / denom if denom != 0 else 0


def weight_calculation(fitness_improvements):
    fitness_improvements = np.abs(fitness_improvements)
    weights = fitness_improvements / np.sum(fitness_improvements)
    return weights

def calculate_selection_probabilities(SR):
    probabilities = np.zeros_like(SR)
    total_SR = np.sum(SR)
    if total_SR > 0:
        probabilities = SR / total_SR
    return probabilities


def sample_from_archive_with_index(archive, probabilities):
    index = np.random.choice(len(archive), p=probabilities)
    value = archive[index]
    return value, index


In [ ]:
def differential_evolution(fitness_func, dim_num, lw_bound, up_bound, pop_size, min_pop_size, fes_max, archive_size):

  #initlize population
  population = np.random.rand(pop_size, dim_num)
  population = [lw_bound + (x * (up_bound-lw_bound)) for x in population]
  fes = 0
  MCR = np.random.rand(archive_size)
  MF = np.random.rand(archive_size)
  SR = np.ones(archive_size)
  best = np.zeros(dim_num)
  best_fitness = np.inf
  #evaluate
  fitness_scores = [np.sum(fitness_func(x)) for x in population]

  while fes_max >= fes :

    #create triplets
    idx = np.arange(len(population))
    np.random.shuffle(idx)

    if len(population) % 3 != 0:
        extra = len(population) % 3
        idx = idx[:-extra]

    triples = [idx[i:i + 3] for i in range(0, len(idx), 3)]

    SF = []
    SCR = []
    f_improv = []

    for triple in triples:

      prob_MCR = calculate_selection_probabilities(SR)
      prob_MF = calculate_selection_probabilities(SR)

      triple_population = [population[idx] for idx in triple]
      triple_fitness = [fitness_scores[idx] for idx in triple]

      sorted_idx = np.argsort(triple_fitness)
      T_best, T_medium, T_worst = [triple_population[idx] for idx in sorted_idx]
      T_best_idx, T_medium_idx, T_worst_idx = [triple[i] for i in sorted_idx]

      CR_indices = []
      F_indices = []
      CR = []
      F = []

      for _ in range(3):
          cr_val, cr_idx = sample_from_archive_with_index(MCR, prob_MCR)
          f_val, f_idx = sample_from_archive_with_index(MF, prob_MF)
          CR.append(np.random.normal(cr_val, 0.1))
          F.append(np.random.normal(f_val, 0.1))
          CR_indices.append(cr_idx)
          F_indices.append(f_idx)

      sorted_F_indices = np.argsort(F)[::-1]
      sorted_CR_indices = np.argsort(CR)[::-1]

      sorted_F = [F[i] for i in sorted_F_indices]
      sorted_F_indices = [F_indices[i] for i in sorted_F_indices]

      sorted_CR = [CR[i] for i in sorted_CR_indices]
      sorted_CR_indices = [CR_indices[i] for i in sorted_CR_indices]

      CR_worst, CR_medium, CR_best = sorted_CR
      F_worst, F_medium, F_best = sorted_F


      #mutation
      worst_vector = T_worst + F_worst * (T_best - T_worst) + F_worst * (T_medium - T_worst)
      worst_vector = np.clip(worst_vector, lw_bound, up_bound)

      idx1, idx2 = np.random.choice([i for i in range(len(population)) if i != triple[1]], 2, replace=False)
      xr1, xr2 = population[idx1], population[idx2]
      medium_vector = T_medium + F_medium * (T_best - T_medium) + F_medium * (xr1 - xr2)
      medium_vector = np.clip(medium_vector, lw_bound, up_bound)

      idx1, idx2 = np.random.choice([i for i in range(len(population)) if i != triple[0]], 2, replace=False)
      xr1, xr2 = population[idx1], population[idx2]
      best_vector = T_best + F_best * (xr1 - xr2)
      best_vector = np.clip(best_vector, lw_bound, up_bound)


      #crossover
      def crossover(vector, mutant, cr_prob):
        d_rand = np.random.randint(0, dim_num)
        trial_vector = np.zeros(dim_num)
        for j in range(dim_num):
          if np.random.rand() < cr_prob or j == d_rand:
            trial_vector[j] = mutant[j]
          else:
            trial_vector[j] = vector[j]
        return trial_vector

      trial_best = crossover(T_best, best_vector, CR_best)
      trial_medium = crossover(T_medium, medium_vector, CR_medium)
      trial_worst = crossover(T_worst, worst_vector, CR_worst)

      # Selection
      trial_fitness_best = np.sum(fitness_func(trial_best))
      trial_fitness_medium = np.sum(fitness_func(trial_medium))
      trial_fitness_worst = np.sum(fitness_func(trial_worst))

      if trial_fitness_best < fitness_scores[T_best_idx]:

        SF.append(F_best)
        SCR.append(CR_best)
        f_improv.append(trial_fitness_best - fitness_scores[T_best_idx])
        population[T_best_idx] = trial_best
        fitness_scores[T_best_idx] = trial_fitness_best
        SR[F_indices[2]] += 1
        SR[CR_indices[2]] += 1

      if trial_fitness_medium < fitness_scores[T_medium_idx]:

        SF.append(F_medium)
        SCR.append(CR_medium)
        f_improv.append(trial_fitness_best - fitness_scores[T_best_idx])
        population[T_medium_idx] = trial_medium
        fitness_scores[T_medium_idx] = trial_fitness_medium
        SR[F_indices[1]] += 1
        SR[CR_indices[1]] += 1


      if trial_fitness_worst < fitness_scores[T_worst_idx]:

        SF.append(F_worst)
        SCR.append(CR_worst)
        f_improv.append(trial_fitness_best - fitness_scores[T_best_idx])
        population[T_worst_idx] = trial_worst
        fitness_scores[T_worst_idx] = trial_fitness_worst
        SR[F_indices[0]] += 1
        SR[CR_indices[0]] += 1



    #update archives
    if SF :
       weights = weight_calculation(f_improv)
       F_output = weighted_lehmer_mean(SF, weights)
       CR_output = weighted_lehmer_mean(SCR, weights)

       MCR[sorted_CR_indices] = CR_output
       MF[sorted_F_indices] = F_output
       SR = np.ones(len(MF))

    best_fitness  = np.min(fitness_scores)
    best_idx = np.argmin(fitness_scores)
    best = population[best_idx]

    new_pop_size = int(np.round((fes / fes_max) * (min_pop_size - pop_size) + pop_size))
    fes+=1
    if fes > fes_max :
      break

  return best, best_fitness


In [ ]:
import timeit
import numpy as np

# Wrapper function for timeit (we discard the return values)
def run_de_algorithm():
    differential_evolution(sphere_function, 100, -100, 100, 100, 4, 1000, 200)

# Run it 11 times and time each run
times = timeit.repeat(
    stmt='run_de_algorithm()',
    setup='from __main__ import run_de_algorithm, differential_evolution, sphere_function',
    repeat=1,
    number=1
)

# Compute mean and standard deviation
mean_time = np.mean(times)
std_time = np.std(times)

print(f"Mean execution time: {mean_time:.4f} seconds")
print(f"Standard deviation: {std_time:.4f} seconds")


Mean execution time: 28.8400 seconds
Standard deviation: 0.0000 seconds


In [ ]:
import timeit
import numpy as np

# Wrapper function for timeit (we discard the return values)
def run_de_algorithm():
    differential_evolution(sphere_function, 300, -100, 100, 100, 4, 1000, 200)

# Run it 11 times and time each run
times = timeit.repeat(
    stmt='run_de_algorithm()',
    setup='from __main__ import run_de_algorithm, differential_evolution, sphere_function',
    repeat=1,
    number=1
)

# Compute mean and standard deviation
mean_time = np.mean(times)
std_time = np.std(times)

print(f"Mean execution time: {mean_time:.4f} seconds")
print(f"Standard deviation: {std_time:.4f} seconds")


Mean execution time: 46.6190 seconds
Standard deviation: 0.0000 seconds


In [ ]:
import timeit
import numpy as np

# Wrapper function for timeit (we discard the return values)
def run_de_algorithm():
    differential_evolution(sphere_function, 500, -100, 100, 100, 4, 1000, 200)

# Run it 11 times and time each run
times = timeit.repeat(
    stmt='run_de_algorithm()',
    setup='from __main__ import run_de_algorithm, differential_evolution, sphere_function',
    repeat=1,
    number=1
)

# Compute mean and standard deviation
mean_time = np.mean(times)
std_time = np.std(times)

print(f"Mean execution time: {mean_time:.4f} seconds")
print(f"Standard deviation: {std_time:.4f} seconds")


Mean execution time: 64.5042 seconds
Standard deviation: 0.0000 seconds


In [ ]:
import timeit
import numpy as np

# Wrapper function for timeit (we discard the return values)
def run_de_algorithm():
    differential_evolution(sphere_function, 750, -100, 100, 100, 4, 1000, 200)

# Run it 11 times and time each run
times = timeit.repeat(
    stmt='run_de_algorithm()',
    setup='from __main__ import run_de_algorithm, differential_evolution, sphere_function',
    repeat=1,
    number=1
)

# Compute mean and standard deviation
mean_time = np.mean(times)
std_time = np.std(times)

print(f"Mean execution time: {mean_time:.4f} seconds")
print(f"Standard deviation: {std_time:.4f} seconds")


Mean execution time: 79.4106 seconds
Standard deviation: 0.0000 seconds


In [ ]:
import timeit
import numpy as np

# Wrapper function for timeit (we discard the return values)
def run_de_algorithm():
    differential_evolution(sphere_function, 1000, -100, 100, 100, 4, 1000, 200)

# Run it 11 times and time each run
times = timeit.repeat(
    stmt='run_de_algorithm()',
    setup='from __main__ import run_de_algorithm, differential_evolution, sphere_function',
    repeat=1,
    number=1
)

# Compute mean and standard deviation
mean_time = np.mean(times)
std_time = np.std(times)

print(f"Mean execution time: {mean_time:.4f} seconds")
print(f"Standard deviation: {std_time:.4f} seconds")


Mean execution time: 100.5655 seconds
Standard deviation: 0.0000 seconds


In [ ]:
import timeit
import numpy as np

# Wrapper function for timeit (we discard the return values)
def run_de_algorithm():
    differential_evolution(sphere_function, 3000, -100, 100, 100, 4, 1000, 200)

# Run it 11 times and time each run
times = timeit.repeat(
    stmt='run_de_algorithm()',
    setup='from __main__ import run_de_algorithm, differential_evolution, sphere_function',
    repeat=1,
    number=1
)

# Compute mean and standard deviation
mean_time = np.mean(times)
std_time = np.std(times)

print(f"Mean execution time: {mean_time:.4f} seconds")
print(f"Standard deviation: {std_time:.4f} seconds")


Mean execution time: 258.9708 seconds
Standard deviation: 0.0000 seconds


In [ ]:
import timeit
import numpy as np

# Wrapper function for timeit (we discard the return values)
def run_de_algorithm():
    differential_evolution(sphere_function, 5000, -100, 100, 100, 4, 1000, 200)

# Run it 11 times and time each run
times = timeit.repeat(
    stmt='run_de_algorithm()',
    setup='from __main__ import run_de_algorithm, differential_evolution, sphere_function',
    repeat=1,
    number=1
)

# Compute mean and standard deviation
mean_time = np.mean(times)
std_time = np.std(times)

print(f"Mean execution time: {mean_time:.4f} seconds")
print(f"Standard deviation: {std_time:.4f} seconds")


Mean execution time: 416.9080 seconds
Standard deviation: 0.0000 seconds


In [ ]:
import timeit
import numpy as np

# Wrapper function for timeit (we discard the return values)
def run_de_algorithm():
    differential_evolution(sphere_function, 7500, -100, 100, 100, 4, 1000, 200)

# Run it 11 times and time each run
times = timeit.repeat(
    stmt='run_de_algorithm()',
    setup='from __main__ import run_de_algorithm, differential_evolution, sphere_function',
    repeat=1,
    number=1
)

# Compute mean and standard deviation
mean_time = np.mean(times)
std_time = np.std(times)

print(f"Mean execution time: {mean_time:.4f} seconds")
print(f"Standard deviation: {std_time:.4f} seconds")


Mean execution time: 605.2079 seconds
Standard deviation: 0.0000 seconds


In [ ]:
import timeit
import numpy as np

# Wrapper function for timeit (we discard the return values)
def run_de_algorithm():
    differential_evolution(sphere_function, 10000, -100, 100, 100, 4, 1000, 200)

# Run it 11 times and time each run
times = timeit.repeat(
    stmt='run_de_algorithm()',
    setup='from __main__ import run_de_algorithm, differential_evolution, sphere_function',
    repeat=1,
    number=1
)

# Compute mean and standard deviation
mean_time = np.mean(times)
std_time = np.std(times)

print(f"Mean execution time: {mean_time:.4f} seconds")
print(f"Standard deviation: {std_time:.4f} seconds")


Mean execution time: 805.2246 seconds
Standard deviation: 0.0000 seconds
